In [6]:
# 랭체인에서 RAG(검색 + 생성) 구현, 분기체인(규칙위반 멀티체인 선택)
!pip install langchain  langchain-google-genai sentence-transformers python-dotenv langchain-chroma langchain-community

 * LangChain이라는 강력한 프레임워크를 사용하여 인공지능(AI) 모델을 활용하는 두 가지 핵심 기술 (검색 증강 생성(RAG)과 모델의 창의성 제어)

1) LangChain을 활용하여 대규모 언어 모델(LLM)인 Gemini와 벡터 데이터베이스를 연결하는 방법.

  1-1) 검색 증강 생성 (RAG) 구현: 자체 문서를 LLM에 연결하여, 모델이 학습하지 않은 최신 정보나 내부 기밀 정보를 바탕으로 정확한 답변을 생성하도록 하는 기술.
RAG 구조는 문서 저장 → 검색(Retrieval) → 답변 생성(Generation)의 흐름
  
  1-2)멀티 실행 체인 및 Temperature 활용: LLM을 호출할 때 temperature라는 매개변수를 조정하여, 논리적이고 안정적인 답변과 창의적이고 다양한 답변을 동시에 얻어내는 방법을 실습

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document                     # 랭체인에서 문서를 다루는 기본 형식
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma               # 벡터 DB, 문서를 숫자로 변환하여 저장
from langchain_community.embeddings import HuggingFaceEmbeddings  # 임베딩 모델 (문서를 벡터로 변환)
import os
from dotenv import load_dotenv
from google.colab import userdata
load_dotenv()
os.environ["GOOGLE_GENAI_PROJECT"] = userdata.get('GOOGLE_GENAI_PROJECT')

# LLM 모델
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, google_api_key=userdata.get('GOOGLE_API_KEY'))
# 임베딩 모델
# (과금)
# embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

#모델이 문서를 '벡터'라는 숫자 형태로 변환해 Chroma DB에 저장
# 과금없는 모델
embedding_model = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)

# 문서 데이터 준비
# 랭체인 RAG(Retrieval Augmented Generation_검색증강생성)구조는 문서들을 저장 -> 검색 -> 답변 생성하는 흐름을 갖음
# 파일(.txt, .pdf...)을 읽으면 구조가 복잡해 일단 메모리에 문서데이터를 만든다. (document type)
docs = [
    Document(page_content="박 검사장은 지난 19일 법무부 검찰 고위 간부 인사를 통해 서울중앙지검장으로 새로 임명됐다. 정진우 전 서울중앙지검장이 대장동 항소 포기 사태 이후 사의를 표한 지 11일 만이었다.박 검사장은 당시 대검찰청 반부패부장으로서 항소 포기 결정 과정에 깊이 관여했다. 박 검사장은 대장동 1심 선고 이후 법무부 측으로부터 '신중 검토 필요' 의견을 전달받은 뒤, 항소한다는 입장이던 서울중앙지검 수사팀에 재검토를 지휘한 것으로 전해졌다. 당시 수사팀은 박 검사장의 지휘를 사실상 '항소 불허'로 받아들였다는 입장이다. 실제 정진우 전 검사장은 사의를 표명하면서 \"대검의 지휘를 수용하지만, 중앙지검의 의견이 다르다는 점을 명확히 한다\"고 언급한 바 있다.검찰 안팎에서는 항소 포기 사태 지휘라인에 있던 박 검사장이 대장동 수사·공판팀을 이끌게 된 만큼, 조직 안정 및 장악이 쉽지 않을 것이라는 우려도 제기된다."),
    # Document(page_content="") # 주제가 다르면 document를 따로따로 만들어야 정확도가 올라감
]
# 텍스트를 조각으로 쪼개기 (옵션)

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)

# 랭체인이 지원하는 Chromadb에 저장
db = Chroma.from_documents(split_docs, embedding_model)

# Retriver
# 저장된 DB에서 질문과 가장 유사한 문서 조각을 '검색(Retrieve)'할 수 있는 도구를 만듬
retriver = db.as_retriever()

# PromptTemplate 작성
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template = """
      너는 친절하고 똑똑한 AI어시스턴트야.
      아래 문서 내용을 참조해 나의 질문에 정확하게 답을 해줘.
      문서 내용일 불충분한 경우 '문서에 해당 정보가 없어요'라고 답변해.
      문맥 :
      {context}
      질문 :
      {question}
      답변은 5행 정도만 해줘.
    """
)

# 체인 생성(LCEL방식)
def format_docsFunc(docs):
  # 검색된 document 리스트를 하나의 문자열로 문자열을 합쳐 반환하는 헬퍼함수
  return "\n\n".join(doc.page_content for doc in docs)
# 전체 RAG 흐름을 하나의 파이프라인으로 연결합니다. (LangChain Expression Language)
# 질문(question)이 들어오면, Retriever를 통해 문맥(context)을 생성하고, 이를 Prompt에 넣어 LLM을 호출하는 구조
rag_chain = (
    {
        "context":retriver | format_docsFunc, #질문 -> retriver로 검색 -> 문서들을 문자열로 포맷
        "question":RunnablePassthrough()      #원래 질문은 그대로 전달
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

# 질문
query = "검찰청에 무슨일이 있었던거야?"
result = rag_chain.invoke(query)
print('질문 : ', query)
print('답변 : ', result)

질문 :  검찰청에 무슨일이 있었던거야?
답변 :  검찰청에서는 다음과 같은 일이 있었습니다.

1.  박 검사장이 법무부 검찰 고위 간부 인사를 통해 서울중앙지검장으로 새로 임명되었습니다.
2.  이는 정진우 전 서울중앙지검장이 대장동 항소 포기 사태 이후 사의를 표명한 지 11일 만이었습니다.
3.  박 검사장은 당시 대검찰청 반부패부장으로서 대장동 1심 선고 후 서울중앙지검 수사팀에 항소 재검토를 지휘했고, 이는 사실상 '항소 불허'로 받아들여졌습니다.
4.  이에 정진우 전 지검장은 대검의 지휘를 수용하지만 중앙지검의 의견이 다르다는 점을 명확히 하며 사의를 표했습니다.
5.  검찰 안팎에서는 항소 포기 사태 지휘라인에 있던 박 검사장이 대장동 수사·공판팀을 이끌게 된 만큼 조직 안정 및 장악이 쉽지 않을 것이라는 우려가 제기됩니다.


In [8]:
# LLM의 답변 성향을 제어하는 핵심적인 방법
# 멀티 실행 체인  : RunnalbeParallel을 사용해 여러 체인을 병렬로 처리
# 여건상 RunnableParallel 사용 불가. 그래서 여기서는 순차적으로 두 번 LLM 호출
llm_calm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1, google_api_key=userdata.get('GOOGLE_API_KEY'))
llm_creative = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.9, google_api_key=userdata.get('GOOGLE_API_KEY'))
q = "AI가 뭔가요?"
out1 = llm_calm.invoke(q)
out2 = llm_creative.invoke(q)
print("1) 현실적인 답변 : ", out1.content)
print("2) 창의적인 답변 : ", out2.content)


1) 현실적인 답변 :  AI는 **인공지능(Artificial Intelligence)**의 줄임말입니다.

가장 쉽게 설명하자면, **컴퓨터가 사람처럼 생각하고, 배우고, 문제를 해결하고, 의사결정을 내리도록 만드는 기술**이라고 할 수 있습니다.

**핵심적인 특징은 다음과 같습니다:**

1.  **학습 (Learning):** 데이터를 분석하여 패턴을 찾고, 그 패턴을 바탕으로 스스로 지식을 습득하고 성능을 향상시킵니다. (예: 많은 고양이 사진을 보고 '고양이'를 인식하는 법을 배우는 것)
2.  **추론 및 문제 해결 (Reasoning & Problem Solving):** 주어진 정보를 바탕으로 논리적으로 생각하고, 특정 목표를 달성하기 위한 최적의 방법을 찾아냅니다. (예: 바둑이나 체스에서 다음 수를 예측하고 승리 전략을 세우는 것)
3.  **지각 (Perception):** 시각(이미지, 영상), 청각(음성) 등 외부 환경의 정보를 인식하고 이해합니다. (예: 얼굴 인식, 음성 인식 비서)
4.  **언어 이해 및 생성 (Language Understanding & Generation):** 사람의 언어를 이해하고, 그에 맞는 답변이나 글을 생성합니다. (예: 챗봇, 번역기)
5.  **의사결정 (Decision Making):** 학습된 지식과 추론을 바탕으로 특정 상황에서 가장 적절한 결정을 내립니다. (예: 자율주행차가 도로 상황에 맞춰 주행 방향을 결정하는 것)

**AI는 우리 주변의 다양한 곳에서 활용되고 있습니다:**

*   **스마트폰:** 음성 비서(시리, 빅스비), 얼굴 인식 잠금 해제, 사진 자동 분류
*   **인터넷:** 검색 엔진, 추천 시스템(넷플릭스, 유튜브, 쇼핑몰), 스팸 메일 필터링
*   **자동차:** 자율주행, 운전자 보조 시스템
*   **의료:** 질병 진단 보조, 신약 개발
*   **로봇:** 산업용 로봇, 서비스 로봇
*   **금융:** 사기 탐지, 주식 시장 예측
*   **챗봇:**

 1. RAG의 '데이터 처리' 핵심 요소 깊이 이해하기

     RAG 체인의 성공은 LLM 성능 20%, 나머지 80%는 데이터를 어떻게 처리했는지에 달려 있습니다. 특히 임베딩, 분할, 벡터 저장소의 역할에 집중.

        1-1) 임베딩 모델 (Embedding Model)의 역할
              HuggingFaceEmbeddings이 하는 역할은 단순한 변환이 아님.
              
              배워야 할 세부 내용:"벡터란 무엇인가?" 문장의 의미를 수백 차원의 숫자 배열로 표현하는 방법.
              임베딩의 품질: 왜 특정 임베딩 모델(예: all-MiniLM-L6-v2)을 선택했는지, 다른 모델을 사용하면 결과가 어떻게 달라지는지 이해해야 함. 임베딩 모델은 의미론적 유사성을 얼마나 잘 포착하느냐가 핵심.
              데이터 업무 연결: 기업의 데이터(업무 보고서, 규정집 등)를 분석할 때, 이 임베딩 모델이 데이터 간의 관계를 정의하는 가장 중요한 도구.

        1-2) 텍스트 분할기 (Text Splitter)의 중요성 CharacterTextSplitter(chunk_size=200, chunk_overlap=20)를 사용했는데 이 숫자가 검색 품질을 결정.
        
              배워야 할 세부 내용:chunk_size의 최적화: 왜 200이 적절한지? 500이나 50으로 바꾸면 어떤 문제가 생길까? 너무 크면 LLM이 불필요한 정보까지 받아 응답이 부정확해지고, 너무 작으면 문맥(Context)이 끊겨 의미 파악이 안 됨.
              chunk_overlap의 역할: 중복되는 20글자(Overlap)는 왜 필요한가요? 이는 분할된 문서 조각(Chunk) 사이의 문맥 연속성을 확보하여, 검색 시 핵심 정보가 조각의 경계에 걸쳐 있더라도 놓치지 않도록!.
              
              다른 분할 전략: 문자를 기준으로 자르는 CharacterTextSplitter 외에, 문단, 문장, 코드를 인식하여 자르는 다른 분할기(예: RecursiveCharacterTextSplitter)가 왜 필요한지 비교하며 학습해야 함.
        
        1-3) 벡터 데이터베이스 (Chroma)의 기능Chroma.from_documents를 사용하여 벡터 DB에 저장.

              
              배워야 할 세부 내용:기존 DB와의 차이점: SQL을 사용하는 관계형 DB(RDB)가 '정확한 값'(예: 이름='홍길동')을 찾는다면, 벡터 DB는 '가장 비슷한 의미'를 갖는 데이터를 찾음.
              데이터 업무를 하려면 두 DB의 특성과 사용 시점을 명확히 구분해야 함.
              검색(Retrieval) 원리: Chroma가 질문 벡터와 저장된 문서 벡터 간의 유사도(Similarity)를 어떻게 측정하는지(예: 코사인 유사도) 원리를 학습
        
  2. RAG 체인을 '데이터 파이프라인'으로 확장하기 RAG 구축 경험은 데이터 엔지니어링의 핵심인 ETL (Extract-Transform-Load) 파이프라인과 매우 유사.
  
          ETL 단계 - RAG 파이프라인 단계 - 학습 연관성
          Extract (추출) - 문서 수집 (Document 생성) - 전산 시스템에서 로그/텍스트 데이터 추출
          Transform (변환) - 텍스트 분할, 임베딩(CharacterTextSplitter, Embedding) - 파이썬 Pandas를 이용한 데이터 정제 및 가공
          Load (적재) - 벡터 DB 저장 (Chroma.from_documents) - SQL을 이용한 전통 DB 또는 벡터 DB에 데이터 적재
    
  
      심화 학습 주제: 데이터 가공 (Transform)RAG 시스템에 넣기 전에 파이썬 Pandas를 사용해서 문서 데이터를 정제하는 방법. (예: 불필요한 특수문자 제거, 중복 문서 제거, 오탈자 수정 등) 깨끗한 데이터가 정확한 RAG 결과를 만든다!
  
  3. LLM 제어 및 활용 심화: Prompt Engineering단순히 temperature를 조절하는 것을 넘어, LLM의 답변 품질과 성향을 명시적으로 제어하는 기술
      1) 프롬프트 엔지니어링 (Prompt Engineering) 심화파일에서 PromptTemplate을 사용했는데 템플릿에 어떤 내용을 넣느냐에 따라 AI의 업무 성과가 결정됨.


          배워야 할 세부 내용:
          
          역할 부여 (Persona): 프롬프트 설정을 특정 업무 역할을 부여하는 연습.
          사고 과정 유도 (Chain-of-Thought, CoT): LLM에게 최종 답변을 내기 전에 "먼저 단계별로 분석하고 그 과정을 나열한 뒤, 최종 답변을 해라"와 같이 사고의 절차를 요구하여 답변의 논리성과 정확성을 높이는 방법으로 학습.
  
      2) LangChain 심화 체인 구조파일에서는 가장 기본적인 RAG 체인을 구성.
          
          배워야 할 세부 내용 :
          
          라우터(Router) 체인: 사용자의 질문을 분석하여, 데이터 분석이 필요한 질문은 RAG 체인으로 보내고, 일반 상식 질문은 일반 LLM으로 바로 보내도록 분기하는 구조를 만들어보기. 이는 복잡한 실무 환경에서 AI의 효율을 극대화하는 방법.
          라우터(Router) 체인은 실무 환경에서 AI 시스템의 효율성과 비용을 획기적으로 개선할 수 있는 매우 중요한 기술임. 마치 AI 시스템의 지능적인 길 안내자

         라우터(Router) 체인이란? : 라우터 체인은 들어오는 사용자의 질문(Query)을 분석하여, 이 질문에 답변하기 위해 가장 적합한 하위 체인(Sub-Chain) 또는 도구(Tool)로 요청을 분기(Routing)시키는 역할을 하는 상위 체인(Master Chain)

            단계별 원리 : 라우팅 LLM 정의: 질문을 분석하고 어떤 경로로 보낼지 결정할 LLM을 정의.
            분기 프롬프트 (Router Prompt): 이 LLM에게 "이 질문은 RAG에 적합한가? 아니면 일반 상식에 적합한가?"를 판단하도록 명시적으로 지시하는 프롬프트를 작성.
            경로 선택 (Route Selection): 라우팅 LLM의 답변(예: "RAG")을 분석하여 실제 실행할 하위 체인을 선택.
            하위 체인 실행: 선택된 하위 체인(RAG 또는 LLM)을 실행


In [12]:
# RAG 체인 재구성을 위한 모듈 및 객체 정의
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()
os.environ["GOOGLE_GENAI_PROJECT"] = userdata.get('GOOGLE_GENAI_PROJECT')

# 1. LLM 정의 (router_llm, normal_llm_chain에 사용)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, google_api_key=userdata.get('GOOGLE_API_KEY'))

# 2. 임베딩 및 DB 정의
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
docs = [Document(page_content="박 검사장은 지난 19일 법무부 검찰 고위 간부 인사를 통해 서울중앙지검장으로 새로 임명됐다. 그는 대장동 개발사업 의혹의 수사를 이끌었다. 다만, 조직 안정 및 장악이 쉽지 않을 것이라는 우려도 제기된다.")]
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)
db = Chroma.from_documents(split_docs, embedding_model)
retriver = db.as_retriever() # 검색기 정의

# 3. format_docsFunc 정의 (검색된 문서를 문자열로 합치기)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 4. RAG 프롬프트 정의
rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template = "아래 문서 내용을 참조해 나의 질문에 정확하게 답을 해줘. 문서 내용이 불충분한 경우 '문서에 해당 정보가 없어요'라고 답변해. 문맥 : {context} 질문 : {question}"
)

# 5. RAG 체인 정의
rag_chain = ({
    "context": (lambda x: x["question"]) | retriver | format_docs, # 질문을 retriever에 전달하여 문맥 생성
    "question": RunnablePassthrough()                             # 원래 질문은 그대로 전달
}
| rag_prompt
| llm
| StrOutputParser())

# 6. 일반 LLM 체인 정의 (일반 상식 질문 처리)
# 수정: 입력 딕셔너리에서 'question' 키의 값을 추출하도록 변경
normal_llm_chain = (lambda x: x['question']) | llm | StrOutputParser()

# 7. 라우터 프롬프트 정의
router_prompt = PromptTemplate.from_template("""
주어진 질문이 내부 문서를 참조하여 답변해야 하는 질문인지,
또는 일반적인 상식에 대한 질문인지 판단하여 해당하는 체인의 이름을 출력해주세요.
내부 문서를 참조해야 하는 질문이라면 "RAG_CHAIN",
일반적인 상식에 대한 질문이라면 "NORMAL_LLM" 이라고 정확히 출력해주세요.

질문: {question}
""")

# 8. 라우터 체인 정의
routing_chain = (
    {"question": RunnablePassthrough()} # 질문을 그대로 전달
    | router_prompt
    | llm
    | StrOutputParser()
)

# 9. 최종 라우터 체인 구성
final_router_chain = RunnablePassthrough.assign(
    route_info = routing_chain # 질문이 들어오면 먼저 라우팅 판단을 하여 route_info 변수에 저장
) | RunnableBranch(
    # 조건식: route_info의 결과가 "RAG_CHAIN" 인가?
    (lambda x: x['route_info'] == "RAG_CHAIN", rag_chain),
    # 조건식: route_info의 결과가 "NORMAL_LLM" 인가?
    (lambda x: x['route_info'] == "NORMAL_LLM", normal_llm_chain),
    normal_llm_chain # Fallback: 위에 해당하지 않으면 일반 LLM 실행
)

# 10. 실행 코드 (Invocation)
# RAG 체인으로 가야 하는 질문 (내부 문서 관련)
rag_query = "박 검사장은 언제 서울중앙지검장으로 새로 임명되었나요?"

# 일반 LLM 체인으로 가야 하는 질문 (일반 상식)
normal_query = "인공지능(AI)이란 무엇인가요?"


print("--- 1. RAG Query 실행 (기대 경로: RAG_CHAIN) ---")
rag_result = final_router_chain.invoke({'question': rag_query})
print(f"경로 판단 결과: RAG_CHAIN으로 예상")
print(f"질문: {rag_query}")
print(f"답변: {rag_result}")
print("--------------------------------------------------")

print("--- 2. Normal LLM Query 실행 (기대 경로: NORMAL_LLM) ---")
normal_result = final_router_chain.invoke({'question': normal_query})
print(f"경로 판단 결과: NORMAL_LLM으로 예상")
print(f"질문: {normal_query}")
print(f"답변: {normal_result}")
print("--------------------------------------------------")

--- 1. RAG Query 실행 (기대 경로: RAG_CHAIN) ---
경로 판단 결과: RAG_CHAIN으로 예상
질문: 박 검사장은 언제 서울중앙지검장으로 새로 임명되었나요?
답변: 박 검사장은 지난 19일 서울중앙지검장으로 새로 임명되었습니다.
--------------------------------------------------
--- 2. Normal LLM Query 실행 (기대 경로: NORMAL_LLM) ---
경로 판단 결과: NORMAL_LLM으로 예상
질문: 인공지능(AI)이란 무엇인가요?
답변: 인공지능(AI)은 **인간의 지능을 모방하고 구현하는 기술 또는 시스템**을 의미합니다.

좀 더 자세히 설명하자면, AI는 컴퓨터 시스템이 다음과 같은 인간의 인지적 능력을 수행할 수 있도록 설계된 분야입니다:

1.  **학습 (Learning):** 경험을 통해 데이터를 분석하고 패턴을 인식하여 스스로 지식을 습득합니다. (예: 방대한 사진을 보고 고양이와 개를 구분하는 방법을 배우는 것)
2.  **추론 (Reasoning):** 학습된 지식과 규칙을 바탕으로 논리적인 결론을 도출하거나 문제를 해결합니다. (예: 특정 증상들을 바탕으로 질병을 진단하는 것)
3.  **인지 (Perception):** 시각, 청각 등 감각을 통해 정보를 받아들이고 이해합니다. (예: 음성 명령을 이해하거나 사진 속 사물을 인식하는 것)
4.  **문제 해결 (Problem Solving):** 주어진 목표를 달성하기 위한 최적의 방법을 찾아냅니다. (예: 바둑이나 체스에서 다음 수를 결정하는 것)
5.  **언어 이해 및 생성 (Language Understanding & Generation):** 인간의 언어를 이해하고 자연스러운 언어를 만들어냅니다. (예: 번역 앱, 챗봇)
6.  **의사결정 (Decision Making):** 여러 대안 중에서 가장 합리적인 선택을 합니다. (예: 자율주행차가 경로를 결정하는 것)

**핵심 요약:**

* 

In [14]:
print('\n\n분기 처리(조건에 따라 체인을 선택)')
from langchain_core.runnables import RunnableBranch, RunnableLambda

print('RunnableBranch 이해 : RunnableBranch((조건, 체인1), 체인2)')
def is_weather_question(text:str) -> bool:
  return "날씨" in text.lower()

# 분기 a
weather_chain = RunnableLambda(lambda x : f"오늘의 날씨는 흐리고 기온은 12도 입니다.")

# 분기 b
general_chain = RunnableLambda(lambda x : f"일반적인 질문이군요. '{x}'에 대해 설명할게요 ")

# 분기 조합
branch_chain = RunnableBranch((is_weather_question, weather_chain), general_chain)

# 실행
print("날씨 질문 테스트 : ", branch_chain.invoke("오늘 날씨는 어때?"))
print("일반 질문 테스트 : ", branch_chain.invoke("Ai란 뭐니?"))



분기 처리(조건에 따라 체인을 선택)
RunnableBranch 이해 : RunnableBranch((조건, 체인1), 체인2)
날씨 질문 테스트 :  오늘의 날씨는 흐리고 기온은 12도 입니다.
일반 질문 테스트 :  일반적인 질문이군요. 'Ai란 뭐니?'에 대해 설명할게요 


In [16]:
print('\n\n분기 처리(LLM 적용)')
llm2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, google_api_key=userdata.get('GOOGLE_API_KEY'))
from langchain_core.output_parsers import StrOutputParser

# prompt 생성 함수들 정의 : 질문을 그대로 LLM에 던지지않고 역할/형식을 지정한 후 질문하기

# 수학 질문용 프롬프트
def make_math_prompt(question:str) -> str:
  return(
      "너는 수학 풀이를 잘하는 모범생이야. \n"
      "아래 수학 문제를 단계별로 풀어보고, 마지막 줄에 정답만 한번 적어줘. \n"
      f"문제: {question}"
      "풀이 : "
  )

# 코딩 질문용 프롬프트
def make_code_prompt(question:str) -> str:
  return(
      "너는 친절한 프로그래밍 전문가야. \n"
      "아래 요청에 대해, 코드를 작성해줘  \n"
      f"질문: {question}\n\n"
      "답변 : "
  )



# 일반 질문용 프롬프트
def make_general_prompt(question:str) -> str:
  return(
      "너는 유능한 데이터 과학자야. \n"
      "아래 질문에 대해 초보자도 이해할 수 있도록 5~6행 문장으로 설명해 줘. \n"
      f"질문: {question}\n\n"
      "답변 : "
  )

# 각 체인 구성(LCEL 방식 : 입력 -> 프롬프트 -> LLM -> 결과 출력)
math_chain = (
    RunnableLambda(make_math_prompt)
    | llm2
    | StrOutputParser()
)
code_chain = (
    RunnableLambda(make_code_prompt)
    | llm2
    | StrOutputParser()
)
gen_general_chain = (
    RunnableLambda(make_general_prompt)
    | llm2
    | StrOutputParser()
)

# 분기 조건 (워크플로우)
def is_math_question(text:str) -> bool:
  # 수학 관련 키워드 / 기호가 있으면 수학 질문으로 간주
  t = text.replace("", "").lower()
  math_keywords = ["더하기","빼기","곱하기","나누기","게산","합","차","곱"]
  math_symblos = ["+","-", "*", "/", "^"]
  return any( k in t for k in math_keywords) or any(s in t for s in math_symblos)


def is_code_question(text:str) -> bool:
  # 프로그래밍 관련 키워드 / 기호가 있으면 코딩 질문으로 간주
  t = text.lower()
  code_keywords = ["코드", "함수", "클래스", "메소드", "알고리즘", "python", "java", "c언어"]
  return any(k in t for k in code_keywords)

# 분기 처리 체인
branch_chain = RunnableBranch(
    (is_math_question, math_chain),
    (is_code_question, code_chain),
    gen_general_chain
)

# 질문하기
q1 = "3 더하기 5 곱하기 2는 얼마인가?"
print('\n수학 질문 연습 : ', q1)
print("결과 1 :" , branch_chain.invoke(q1).strip())

q2 = "파이썬으로 숫자들의 평균을 구하는 예제를 만들어"
print('\n코딩 질문 연습 : ', q2)
print("결과 2 :" , branch_chain.invoke(q2).strip())

q3 = "가을과 겨울의 차이를 설명해"
print('\n일반 질문 연습 : ', q3)
print("결과 3 :" , branch_chain.invoke(q3).strip())




분기 처리(LLM 적용)

수학 질문 연습 :  3 더하기 5 곱하기 2는 얼마인가?
결과 1 : 네, 문제 풀어보겠습니다.

주어진 문제는 "3 더하기 5 곱하기 2" 입니다.

**단계별 풀이:**

1.  **연산 순서 파악:** 수학에서는 곱셈과 나눗셈을 덧셈과 뺄셈보다 먼저 계산합니다. 따라서 이 문제에서는 곱셈을 먼저 계산해야 합니다.
2.  **곱셈 계산:** 먼저 5와 2를 곱합니다.
    $5 \times 2 = 10$
3.  **덧셈 계산:** 이제 곱셈의 결과(10)에 3을 더합니다.
    $3 + 10 = 13$

정답: 13

코딩 질문 연습 :  파이썬으로 숫자들의 평균을 구하는 예제를 만들어
결과 2 : 안녕하세요! 숫자들의 평균을 구하는 것은 데이터 분석의 가장 기본적인 단계 중 하나입니다.

평균은 모든 숫자를 더한 후, 그 숫자들의 총 개수로 나눈 값입니다. 파이썬에서는 이 계산을 아주 쉽게 할 수 있습니다. 먼저 숫자들을 리스트(list)에 담고, 내장 함수인 `sum()`으로 합계를, `len()`으로 개수를 얻습니다. 이 둘을 나누면 간단히 평균을 구할 수 있습니다. 예를 들어, `[10, 20, 30]`의 평균을 구하는 코드는 다음과 같습니다.

```python
numbers = [10, 20, 30]
total = sum(numbers)       # 숫자들의 합계 (10 + 20 + 30 = 60)
count = len(numbers)       # 숫자들의 개수 (3개)
average = total / count    # 합계를 개수로 나눔 (60 / 3 = 20.0)

print(f"숫자들의 평균: {average}") # 결과: 숫자들의 평균: 20.0
```

일반 질문 연습 :  가을과 겨울의 차이를 설명해
결과 3 : ## 풀이 :

**1단계: 문제 이해 및 용어 정의**
본 문제는 '가을'과 '겨울'이라는 두 계절의 주요 차이점을 명확하게 설명하는 것입니다. 두 계절의 일반적인 특징을 먼저 정의